In [1]:
import keras
import tensorflow as tf
import pyfaidx
import math
import pandas as pd
import tqdm
import numpy as np
import pyBigWig
import scipy
import sys
import os
from keras.utils import CustomObjectScope
from modisco.visualization import viz_sequence
import keras.backend as K
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
hg38 = pyfaidx.Fasta('/mnt/data/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta')
bw = pyBigWig.open("/srv/scratch/anusri/chrombpnet_paper/results/chrombpnet/ATAC_PE/GM12878/data/shifted.sorted.bam.chrombpnet.unstranded.bw") # +4/-5 shifted
gc_neg_peaks =  pd.read_csv("/srv/scratch/anusri/chrombpnet_paper/results/chrombpnet/ATAC_PE/GM12878/negatives_data/negatives.bed", # these are non-peaks for fibroblast scATAC pseudo-bulk
                           sep='\t',
                           names=["chr", "start", "end", "1", "2", "3", "4", "5", "6", "summit"])
print(gc_neg_peaks.shape)
gc_neg_peaks["summit"] = (gc_neg_peaks["start"] + gc_neg_peaks["end"]) // 2
gc_neg_peaks.head()

(189914, 10)


,chr,start,end,1,2,3,4,5,6,summit
0,chr1,76592500,76594614,0.38,NaN,NaN,NaN,NaN,NaN,76593557
1,chr1,162533550,162535664,0.46,NaN,NaN,NaN,NaN,NaN,162534607
2,chr1,175032450,175034564,0.45,NaN,NaN,NaN,NaN,NaN,175033507
3,chr1,151808900,151811014,0.46,NaN,NaN,NaN,NaN,NaN,151809957
4,chr1,167826000,167828114,0.45,NaN,NaN,NaN,NaN,NaN,167827057


In [3]:
#sys.path.append("/srv/scratch/anusri/chrombpnet_paper/src/training/utils/") 
#from importlib import reload
#import one_hot

In [4]:
peaks = gc_neg_peaks[(gc_neg_peaks["chr"]=="chr20")]

In [5]:
peaks.shape[0]

3884

In [6]:
def get_seq(genome, peaks_df, width=2000):
    vals = []
    for i, r in peaks_df.iterrows():
        seq = str(genome[r['chr']][(r['start'] + r['summit']) - width//2:(r['start'] + r['summit']) + width//2])
        if len(seq) == 2000:
            vals.append(seq)
        else:
            #print(len(seq))
            #print((r['start'] + r['summit'])- width//2, (r['start'] + r['summit']) + width//2)
            pass
        
    return one_hot.dna_to_one_hot(vals)

In [7]:
def get_cts(bw, peaks_df, width=2000):
    vals = []
    for i, r in peaks_df.iterrows():
        try:
            vals.append(np.nan_to_num(bw.values(r['chr'], 
                                                (r['start'] + r['summit'])- width//2,
                                                (r['start'] + r['summit']) + width//2)))
        except:
            #print(i)
            #print((r['start'] + r['summit'])- width//2, (r['start'] + r['summit']) + width//2)
            pass
        
    return np.array(vals)

In [8]:
seqs = get_seq(hg38, peaks)
cnts = get_cts(bw, peaks)

NameError: name 'one_hot' is not defined

In [ ]:
print(seqs.shape) 
print(cnts.shape)

In [ ]:
def get_pwm_bg(seqs, cnts):
    new_seqs = []
    for i in range(cnts.shape[0]):
        for j in range(13,cnts.shape[1]-13):
            if cnts[i,j] > 0:
                new_seqs.append(seqs[i,j-12:j+12,:]* cnts[i,j])
    print(np.array(new_seqs).shape)
    motif = np.sum(new_seqs, axis=0)
    motif = motif/np.sum(motif, axis=-1, keepdims=True)
    bg = np.sum(np.sum(new_seqs, axis=0), axis=0)
    bg = bg/sum(bg)
    return motif, bg

In [ ]:
motif, bg = get_pwm_bg(seqs, cnts)

In [ ]:
viz_sequence.plot_weights(viz_sequence.ic_scale(motif, background=bg))